## Transmission-Distribution Power Flow Co-simulation
This script runs an iterative transmission-distribution power flow. The network is assumed to consist of a single transmission network connected to distribution feeders at each load bus. The number of distribution feeders connected is determined based on the real power load at the bus and the injection of the distribution feeder. In this example, we use the following:
+ Transmission system: 200-bus network (synthetic network for Illinois system..from TAMU)
+ Distribution feeder: IEEE 8500-node feeder.

The metadata1 supplied has the code for making the connection information for T and D. It is created by the supplied MATLAB function makemetadatafile.m. The metadatafile is required by both the transmission and distribution federates to configure the connection between T and D federates. One can vary the number of federates by setting 'nbdry_selected' variable in the code block below. With nbdry_selected=1, only the first boundary bus will be selected. This boundary bus is set to have three distribution federates (the number of federates is based on the active power load at the bus and the injection at the distribution feeder)

In [1]:
import os
import sys
import subprocess
import shlex
import operator

In [18]:
metadatafile='metadata1'

nbdry_selected = 1
# Metadafile having number of boundary buses 
# and the feeder connections at those buses
file = open(metadatafile,"r")
linenum = 1
bdry_buses = []
dist_feeders = {}
for line in file:
    if linenum == 1:
        nbdry_nfeeders = line.rstrip(' \n').split(',')
        nbdry = int(nbdry_nfeeders[0])
        nfeeders = int(nbdry_nfeeders[1])
    elif linenum < nbdry+2:
        if(len(bdry_buses) < nbdry_selected):
            bdry_buses.append(line.rstrip(' \n'))
    else:
        values1 = line.rstrip(' \n')
        values = values1.split(',')
        if values[0] in bdry_buses:
            dist_feeders[values[1]]= int(values[0])  # name:boundary bus       
    linenum = linenum+1
    
file.close()

nbdry=len(bdry_buses)
nfeeders=len(dist_feeders)
print 'nbdry=%d ' %nbdry
print 'nfeeders=%d'%nfeeders
# Create another metadatafile with the selected number of boundary buses and feeders
fp = open('metadatafile',"w")
fp.write("%d,%d\n" % (nbdry,nfeeders))
for bus in bdry_buses:
    print("%d" % int(bus))
    fp.write("%d\n" % int(bus))

for k in dist_feeders:
    fp.write("%d,%s\n" % (dist_feeders[k],k))
    print("%d,%s" % (dist_feeders[k],k))
fp.close()

nfeds = nfeeders+1

print 'nfederates=%d' %nfeds

metadatafile='metadatafile'

nbdry=1 
nfeeders=3
8
8,dcase_tbdry_8_feeder_2
8,dcase_tbdry_8_feeder_3
8,dcase_tbdry_8_feeder_1
nfederates=4


In [22]:
print 'Broker'
broker_args='-nfeds '+str(nfeds)
broker_cmdline='./helicsbroker '+broker_args
broker = shlex.split(broker_cmdline)
print broker_cmdline+'\n'
## Launch broker
subprocess.Popen(broker)

##Launch Transmission federate
print 'Transmission'
netfile='datafiles/case_ACTIVSg200.m'
#print metadatafile
# Launch Transmission federate simulation
pflowhelicst_args_files  ='-netfile '+netfile+' -metadatafile '+metadatafile
pflowhelicst_args=pflowhelicst_args_files
pflowhelicst_cmdline='./PFLOWHELICST '+pflowhelicst_args
print pflowhelicst_cmdline+'\n'
pflowhelicst = shlex.split(pflowhelicst_cmdline)
subprocess.Popen(pflowhelicst)


##Launch distribution federates
fednum=0
dnetfile='datafiles/8500-Node/Master.dss'
for k in dist_feeders:
    fednum = fednum + 1
    print 'D federate '+k
    # Dist. federate 1
    netfile=dnetfile
    dtopic=k
    pflowhelicsd_args = '-netfile '+netfile+' -dtopic '+dtopic
    pflowhelicsd_cmdline='./PFLOWHELICSD '+pflowhelicsd_args
    print pflowhelicsd_cmdline+'\n'
    pflowhelicsd = shlex.split(pflowhelicsd_cmdline)
    subprocess.Popen(pflowhelicsd)

Broker
./helicsbroker -nfeds 4

Transmission
./PFLOWHELICST -netfile datafiles/case_ACTIVSg200.m -metadatafile metadatafile

D federate dcase_tbdry_8_feeder_2
./PFLOWHELICSD -netfile datafiles/8500-Node/Master.dss -dtopic dcase_tbdry_8_feeder_2

D federate dcase_tbdry_8_feeder_3
./PFLOWHELICSD -netfile datafiles/8500-Node/Master.dss -dtopic dcase_tbdry_8_feeder_3

D federate dcase_tbdry_8_feeder_1
./PFLOWHELICSD -netfile datafiles/8500-Node/Master.dss -dtopic dcase_tbdry_8_feeder_1

